https://mcyoung.xyz/2021/06/01/linker-script/

https://archfx.github.io/posts/2023/02/firmware3/

I've had this fun idea for a while that linker script could be programs.





In [50]:
%%file /tmp/test.c
#include <stdio.h>
int biz;
int *foo, *bar;

int main(){
    foo = &biz;
    bar = &biz;
    *foo = 3;
    printf("%d\n", *bar);
}


Overwriting /tmp/test.c


In [51]:
! gcc -O3 -fanalyzer -fsanitize=undefined -Wall -Wextra /tmp/test.c -o /tmp/test && /tmp/test

3


In [25]:
! cbmc /tmp/test.c

CBMC version 6.0.1 (cbmc-6.0.1-5-g54c20cdb91) 64-bit x86_64 linux
Type-checking test
Generating GOTO Program
Adding CPROVER library (x86_64)
Removal of function pointers and virtual functions
Generic Property Instrumentation
Starting Bounded Model Checking

** Results:
/tmp/test.c function main
[main.pointer_dereference.1] line 8 dereference failure: pointer NULL in *foo: SUCCESS
[main.pointer_dereference.2] line 8 dereference failure: pointer invalid in *foo: SUCCESS
[main.pointer_dereference.3] line 8 dereference failure: deallocated dynamic object in *foo: SUCCESS
[main.pointer_dereference.4] line 8 dereference failure: dead object in *foo: SUCCESS
[main.pointer_dereference.5] line 8 dereference failure: pointer outside object bounds in *foo: SUCCESS
[main.pointer_dereference.6] line 8 dereference failure: invalid integer address in *foo: SUCCESS
[main.pointer_dereference.7] line 9 dereference failure: pointer NULL in *bar: SUCCESS
[main.pointer_dereference.8] line 9 dereference fai

In [32]:
%%file /tmp/test.c
#include <stdio.h>
extern int foo, bar;

int main(){
    foo = 3;
    printf("%d\n", bar);
}


Overwriting /tmp/test.c


In [41]:
! gcc -c -Wall -Wextra /tmp/test.c -o /tmp/test.o

In [47]:
%%file /tmp/linker.ld

SECTIONS
{
    . = 0x1000;  /* Start the text section at 0x1000 */
    .text : {
        *(.text)  /* Place all code in the .text section */
    }
    . = 0x2000;  /* Start the data section at 0x2000 */
    .data : {
        *(.data)  /* Place all initialized data here */
    }
    . = 0x20000;
    foo = .;
    bar = .;
    . += 8; 

}

Overwriting /tmp/linker.ld


In [48]:
! gcc /tmp/test.o -T /tmp/linker.ld -o /tmp/test && /tmp/test

3


In [46]:
! objdump -x /tmp/test | grep 'foo\|bar'

0000000000020000 g       .bss	0000000000000000              bar
0000000000020000 g       .bss	0000000000000000              foo


In [26]:
%%file /tmp/ldscript
SECTIONS
{
  . = 0x10000;
  .text : { *(.text) }
  . = 0x8000000;
  .data : { *(.data) }
  .bss : { *(.bss) }
}


Writing /tmp/ldscript


In [ ]:
! ld -T /tmp/ldscript /tmp/test.o -o /tmp/test && objdump -x /tmp/test | grep 'start address'